In [1]:
import random
import itertools
import threading
import queue
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd
import speaker_buddy as buddy

### Exchanges

In [2]:
EXCHANGES= [
    'bitfinex',
    'coinbase-pro',
    'bitstamp',
    'kraken',
    'cexio',
    'okcoin',
    'bitmex',
    'mexbt',
    'huobi',
    'poloniex',
    'bittrex',
    'okex',
    'hitbtc'
]

### Symbols

In [3]:
SYMBOLS = ['btc', 'ltc', 'eth']

### Combinations

In [4]:
ALL_COMBINATIONS = list(itertools.product(EXCHANGES, SYMBOLS))

### Random Combinatios

In [5]:
combinations = [random.choice(ALL_COMBINATIONS) for _ in range(10)]

### Random Dates

In [6]:
ALL_DATES = [d.strftime('%Y-%m-%d') for d in pd.date_range('2019-01-01', '2019-07-20', freq='D')]

In [7]:
dates = [random.choice(ALL_DATES) for _ in range(5)]

### Summary

In [8]:
dates

['2019-07-10', '2019-06-15', '2019-07-16', '2019-07-20', '2019-07-03']

In [9]:
combinations

[('okcoin', 'ltc'),
 ('kraken', 'ltc'),
 ('bitfinex', 'ltc'),
 ('huobi', 'ltc'),
 ('huobi', 'btc'),
 ('kraken', 'eth'),
 ('bittrex', 'btc'),
 ('mexbt', 'btc'),
 ('bitfinex', 'ltc'),
 ('bitmex', 'btc')]

In [10]:
data = list(itertools.product(combinations, dates))
data[:3]

[(('okcoin', 'ltc'), '2019-07-10'),
 (('okcoin', 'ltc'), '2019-06-15'),
 (('okcoin', 'ltc'), '2019-07-16')]

In [11]:
URL = 'http://localhost:5000/price/{exchange}/{symbol}/{date}'

In [12]:
price_urls = [
        URL.format(exchange=exchange, symbol=symbol, date=date)
    for (exchange, symbol), date in data]

In [13]:
price_urls[:3]

['http://localhost:5000/price/okcoin/ltc/2019-07-10',
 'http://localhost:5000/price/okcoin/ltc/2019-06-15',
 'http://localhost:5000/price/okcoin/ltc/2019-07-16']

In [14]:
len(price_urls)

50

### A simple test

In [15]:
resp = requests.get(price_urls[0])

In [16]:
resp.json()

{'exchange': 'okcoin',
 'symbol': 'ltc',
 'open': 123.26,
 'high': 123.96,
 'low': 118.44,
 'close': 118.71,
 'volume': 879.2801,
 'day': '2019-07-10'}

We'll do the same thing for all of them.

---

# A quick `parallel` example

In [23]:
import parallel

In [27]:
@parallel.decorate(PR)
def get_close_price(url):
    resp = requests.get(url)
    result = resp.json()
    return result['close'] if result else None

In [28]:
get_close_price('http://localhost:5000/price/poloniex/btc/2019-01-13')

3582.2544

Parallalizing this task is a breeze:

In [29]:
prices = get_close_price.map(price_urls, max_workers=4)

In [31]:
prices

[118.71,
 132.79,
 90.57,
 98.15,
 118.57,
 119.38,
 132.68,
 90.59,
 98.78,
 118.55,
 118.92,
 131.24,
 90.896,
 98.421,
 117.44,
 119.22,
 132.09,
 90.38,
 98.25,
 118.82,
 12546.12,
 8649.75,
 10838.53,
 10506.49,
 10843.55,
 307.95,
 263.83,
 228.13,
 221.45,
 291.24,
 12577.848999999998,
 8699,
 10858.695,
 10529.999,
 10841.149,
 None,
 None,
 None,
 None,
 None,
 118.92,
 131.24,
 90.896,
 98.421,
 117.44,
 None,
 None,
 None,
 None,
 None]